In [2]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("YelpDataAnalysis1").getOrCreate()

# Load Yelp dataset (adjust the path if needed)
df = spark.read.json("DA_project.json")

# Show first few rows
df.show(5)


+--------------------------+--------------------+-----------+-----------+----------------------+-----------+-----------+-----------+-----------+-----------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|BusinessAcceptsCreditCards|     BusinessParking|     Friday|     Monday|RestaurantsPriceRange2|   Saturday|     Sunday|   Thursday|    Tuesday|  Wednesday|             address|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------------+--------------------+-----------+-----------+----------------------+-----------+-----------+-----------+-----------+-----------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+---

In [5]:
from elasticsearch import Elasticsearch, helpers
import json

# Connect to Elasticsearch
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "elvaelastic")
)

# Load cleaned data
with open("DA_Project.json", "r") as f:
    try:
        # Try loading as a JSON array (standard format)
        data = json.load(f)
    except json.JSONDecodeError:
        # If that fails, try loading it as NDJSON (line-separated JSON)
        f.seek(0)  # Reset file pointer
        data = [json.loads(line) for line in f]

# Define index name
index_name = "yelp_data_cleaned1"

# Check if index exists, if not, create it
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Bulk insert data
actions = [
    {
        "_index": index_name,
        "_source": doc
    }
    for doc in data
]

helpers.bulk(es, actions)
print("Data successfully indexed into Elasticsearch!")

Data successfully indexed into Elasticsearch!
